### CNN10のTranscriptからスクレイピングでページ上の全件の情報を抽出する

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [ ]:
# transcriptの一覧ページのURL
base_url = "http://transcripts.cnn.com/"
aired_list_url = base_url + "TRANSCRIPTS/sn.html"

In [ ]:
res = urllib.request.urlopen(aired_list_url)
soup = BeautifulSoup(res, "lxml")

In [ ]:
link_soup = soup.find_all('div', class_="cnnSectBulletItems") # 各放送のTranscriptのみに範囲を絞る

In [ ]:
# 取得したい項目のリストを初期化
title_list = []
content_list = []
date_list = []

for l in link_soup:
    script_url = l.a["href"] # 各放送日のTranscriptのURLを取得
    title = l.a.text # 各放送のタイトルを取得
    # 取得したURLをパースして放送年月日を文字列で抽出
    # '/TRANSCRIPTS/1912/13/sn.01.html'のような形式からパース
    # 2100年移行には使えなくなるよくないコードだが現時点では許容する
    date = "20" + l.a["href"].split("/")[2] + l.a["href"].split("/")[3] 
    # datetimeを使用してdatetime型に変換
    conv_date = datetime.datetime.strptime(date, '%Y%m%d')
    base_url = "http://edition.cnn.com"
    tar_url = base_url + script_url
    try:
        script_res = urllib.request.urlopen(tar_url)
        script_soup = BeautifulSoup(script_res, "lxml")
        content = script_soup.find_all("p", class_="cnnBodyText")[2].text # うまく本文のみで絞り込めなかった

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
    except : # エラーの際はエラーになったURLを出力して処理は継続 # 404エラーの場合が多い
        print("Error:{}".format(tar_url))

In [ ]:
out_df = pd.DataFrame({"title":title_list, "content":content_list, "date":date_list})
out_df = out_df[["title", "content", "date"]]

In [ ]:
out_df.to_csv("./out/cnn10_scripts_20200101.csv", index=False)